<a href="https://colab.research.google.com/github/mfreeman451/colab/blob/main/Copy_of_GoNB_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoNB - A Go Notebook Kernel for Jupyter

See the [tutorial in github](https://github.com/janpfeifer/gonb/blob/e15ac2e8e3fe/examples/tutorial.ipynb). The repository is in [github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb).



## Installation in Google's Colab

Run the following cell once only. It will install Go, GoNB and restart the kernel so it uses Go instead of the provided Python.

It takes a couple of minutes ... but only needs to be done once.

When creating other GoNB notebooks with Google's Colab, you will have to copy the cell below (or do something similar).

**Disclaimer**: this is highly not documented or official, but seems to be supported by Colab (it works). Likely this way of replacing the kernel will break without notice. Please reach out to project in [github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb) if there are any issues.


In [ ]:
#@title Install Go, `goimports` and Gote code.

# Install Go and goimports.
!mkdir -p cache
!wget -q -O cache/go.tar.gz 'https://go.dev/dl/go1.20.linux-amd64.tar.gz'
!tar xzf cache/go.tar.gz
%env GOROOT=/content/go
!ln -sf "/content/go/bin/go" /usr/bin/go
!go version
!go install golang.org/x/tools/cmd/goimports@latest 2> /dev/null
!ln -sf /root/go/bin/goimports /usr/bin/goimports

# Install GoNB.
!git clone -q https://github.com/janpfeifer/gonb.git
!cd gonb ; go build --buildvcs=false -o gonb . >& /dev/null || echo "Failed to build gonb" 
!cd gonb ; ./gonb --install

# Python code to replace ipython kernel with GoNB.
import os
import ipykernel_launcher

call_kernel_code='''
import os
import sys

connection_file = sys.argv[2]
os
try:
  os.execl("/content/gonb/gonb", "/content/gonb/gonb", "--kernel", connection_file)
finally:
  from ipykernel import kernelapp as app
  app.launch_new_instance()
'''
new_file = '/tmp/gonb_ipykernel_launcher.py'
# old_file = '/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py'
old_file = ipykernel_launcher.__file__
with open(new_file, 'w') as f:
  f.write(call_kernel_code)
os.replace(new_file, old_file)

# Finally kill current kernel to force restart.
import os; import sys; sys.stdout.flush(); os.kill(os.getpid(), 9)


env: GOROOT=/content/go
go version go1.20 linux/amd64
[6544c217] 2023/02/12 03:32:30 Go (gonb) kernel configuration installed in "/root/.local/share/jupyter/kernels/gonb/kernel.json".


The cell above will crash the current kernel after replacing it with GoNB. It's normal. Run it once only, it takes a couple of minutes. After that you will be running a GoNB kernel.

## VulnCheck Demo - Exploits API

We're going to get a list of the most recent exploits from the API
and create some visualizations around the data.


First ...

In [ ]:
!*go get -u github.com/erkkah/margaid@d60b2efd2f5acc5d8fbbe13eaf85f1532e11a2fb

go: added github.com/erkkah/margaid v0.1.1-0.20230128143048-d60b2efd2f5a


In [ ]:
%env VC_API_KEY 35829961415c5256e02d5a65e52e982746400286615b384317ffaf1f34036258

import (
	"encoding/json"
	"fmt"
	"github.com/go-gota/gota/dataframe"
	"github.com/joho/godotenv"
	"io"
	"log"
	"net/http"
	"os"
	"strings"
)

func main() {
	// .env init
	err := godotenv.Load()
	if err != nil {
		log.Println("Error loading .env file")
	}

	_, err = GetVCExploits()
	if err != nil {
		log.Fatal(err)
	}

}

func GetVCAPIURL() string {
	return "https://api.vulncheck.com"
}

func GetVCAPIKey() (string, error) {
	vcAPIKey := os.Getenv("VC_API_KEY")
	if vcAPIKey == "" {
		return "", fmt.Errorf("VC_API_KEY not set")
	}
	return vcAPIKey, nil
}

type VCExploitsResults struct {
	Cve                 string        `json:"cve"`
	DateAdded           string        `json:"date_added"`
	ExploitAvailability string        `json:"exploit_availability"`
	ExploitMaturity     string        `json:"exploit_maturity"`
	Name                string        `json:"name"`
	Refsource           string        `json:"refsource"`
	Tags                []interface{} `json:"tags"`
	URL                 string        `json:"url"`
	ExploitType         string        `json:"exploit_type,omitempty"`
}

type VCExploits struct {
	Index        string              `json:"index"`
	SearchType   string              `json:"search_type"`
	RequestTime  int                 `json:"request_time"`
	ResultsCount int                 `json:"results_count"`
	Results      []VCExploitsResults `json:"results"`
}

func GetVCExploits() (dataframe.DataFrame, error) {
	var vcExploits VCExploits
	body, err := GetVCAPI("v2/recent/exploits")
	if err != nil {
		return dataframe.DataFrame{}, err
	}
	err = json.Unmarshal(body, &vcExploits)
	if err != nil {
		return dataframe.DataFrame{}, err
	}

	var results []VCExploitsResults
	for _, result := range vcExploits.Results {
		results = append(results, result)
	}
	// marshal results to json
	jsonResults, err := json.Marshal(results)

	jsonDf := dataframe.ReadJSON(strings.NewReader(string(jsonResults)))
	fmt.Println(jsonDf)

	return jsonDf, nil
}

func GetVCAPI(request string) ([]byte, error) {
	url := fmt.Sprintf("%s/%s", GetVCAPIURL(), request)

	apiKey, err := GetVCAPIKey()
	if err != nil {
		log.Fatalln(err)
	}
	// add Bearer token
	apiKey, kErr := GetVCAPIKey()
	if kErr != nil {
		return nil, kErr
	}
	req, err := http.NewRequest("GET", url, nil)
	if err != nil {
		return nil, err
	}
	var bearer = "Bearer " + apiKey
	req.Header.Add("Authorization", bearer)

	req.Header.Add("accept", "application/json")
	res, resErr := http.DefaultClient.Do(req)
	if resErr != nil {
		return nil, resErr
	}
	defer func(Body io.ReadCloser) {
		err := Body.Close()
		if err != nil {
			log.Fatalln(err)
		}
	}(res.Body)
	body, bodyErr := io.ReadAll(res.Body)
	if bodyErr != nil {
		return nil, bodyErr
	}

	return body, nil
}


2023/02/12 03:35:39 Error loading .env file


[10x9] DataFrame

    cve            date_added exploit_availability   exploit_maturity ...
 0: CVE-2021-21974 2023-02-10 commercially-available weaponized       ...
 1: CVE-2023-0669  2023-02-10 privately-available    weaponized       ...
 2: CVE-2015-2291  2023-02-10 privately-available    weaponized       ...
 3: CVE-2021-43798 2023-02-10 commercially-available poc              ...
 4: CVE-2022-24990 2023-02-10 privately-available    weaponized       ...
 5: CVE-2022-47966 2023-02-09 publicly-available     poc              ...
 6: CVE-2023-0669  2023-02-09 publicly-available     poc              ...
 7: CVE-2023-0669  2023-02-09 publicly-available     weaponized       ...
 8: CVE-2022-24990 2023-02-09 privately-available    weaponized       ...
 9: CVE-2022-48110 2023-02-09 publicly-available     poc              ...
    <string>       <string>   <string>               <string>         ...

Not Showing: exploit_type <string>, name <string>, refsource <string>, tags <string>,
url <st

# More ... and Help

In [ ]:
%help

GoNB is a Go kernel that compiles and executed on-the-fly Go code. 

When executing a cell, *GoNB* will save the cell contents (except non-Go commands see
below) into a "main.go" file, compile and execute it.

It also saves any global declarations (imports, functions, types, variables, constants)
and reuse them at the next cell execution -- so you can define a function in one
cell, and reuse in the next one. Just the "func main()" is not reused.

A "hello world" example would look like:

	func main() {
		fmt.Printf("Hello world!\n");
	}

But to avoid having to type "func main()" all the time, you can use "%%" and everything
after is wrapped inside a "func main() { ... }". So our revised "hello world" looks like:

	%%
	fmt.Printf("Hello world!\n")


- "init()" functions: since there is always only one definition per function name, 
  it's not possible for each cell to have it's own init() function. Instead GoNB
  converts any function named "init_<my_stuff>()" to "init()" before compili